In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
n_input  = 784
n_output = 10
weights = {
    #[width, height, channel, n_filter]
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.1)),
    #weight of dense layer: 28*28 (maxpooling) -> 14*14 -> 7*7*channel
    'wd1': tf.Variable(tf.random_normal([7*7*128, n_output], stddev=0.1))
}
biases = {
    #plus biases using broad casting.
    'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

def conv_simple(_input, _w, _b):
    #Reshape input
    _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
    #layer1
    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1,1,1,1], padding='SAME')
    _conv2 = tf.nn.bias_add(_conv1, _b['bc1'])
    _conv3 = tf.nn.relu(_conv2)
    _pool1 = tf.nn.max_pool(_conv3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #layer2
    _conv4 = tf.nn.conv2d(_pool1, _w['wc2'], strides=[1,1,1,1], padding='SAME')
    _conv5 = tf.nn.bias_add(_conv4, _b['bc2'])
    _conv6 = tf.nn.relu(_conv5)
    _pool2 = tf.nn.max_pool(_conv6, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #Vectorize
    _dense = tf.reshape(_pool2, [-1, _w['wd1'].get_shape().as_list()[0]])
    #Fully-connected layer
    _out = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'conv2': _conv2, 'conv3': _conv3
        , 'pool1': _pool1, 'conv4': _conv4, 'conv5': _conv5, 'conv6': _conv6
        , 'pool2': _pool2, 'dense': _dense, 'out': _out
    }
    return out

In [ ]:
#input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
#parameters
learning_rate   = 0.001
training_epochs = 5
batch_size      = 100
display_step    = 1
#function
_pred = conv_simple(x, weights, biases)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
_corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))
